In [0]:
# Code adapted from https://www.kaggle.com/kanncaa1/recurrent-neural-network-with-pytorch

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
# print(os.listdir("../input"))
import io

from google.colab import files
uploaded = files.upload()
print(type(uploaded))

# Any results you write to the current directory are saved as output.

In [0]:
# Import Libraries
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import json
from sklearn.feature_extraction import DictVectorizer

In [0]:
def encode_labels(y):
  y_names = []
  y_encoded = []
  for label in y:
    if label not in y_names:
      y_names.append(label)
  for i in range(len(y)):
    label = y_names.index(y[i])
    y_encoded.append(label)
  return y_encoded

In [0]:
# Prepare Dataset
# load data
# train = pd.read_csv(r"../input/train.csv",dtype = np.float32)

import random

X = []
y = []
v = DictVectorizer(sparse=False)

for fname in uploaded.keys():
  print(fname)
  if fname.endswith("data.txt"):
    data = []
    with open(fname, 'rb') as f:
      for line in f:
        data.append(json.loads(line))
      X.extend(data)
      # for lis in v.fit_transform(data):
      #   print(len(lis))
    with open(fname.replace("data.txt", "labels.txt")) as f:
      data = f.read().splitlines()
      y.extend(data)
y = encode_labels(y)


X = v.fit_transform(X)
input_dim = X.shape[1]
seq_len = 10
X = np.asarray(X, dtype=np.float32)
y = np.asarray(y)
print(y.shape)
print(X.shape)
batch_size = 10

targets_numpy = y
features_numpy = X / np.amax(X) # normalization
targets_numpy = targets_numpy.reshape(-1, seq_len)
features_numpy = features_numpy.reshape(-1, seq_len, input_dim)
print(targets_numpy.shape)
print(features_numpy.shape)

# train test split. Size of train data is 80% and size of test data is 20%. 
features_train, features_test, targets_train, targets_test = train_test_split(features_numpy,
                                                                             targets_numpy,
                                                                             test_size = 0.2,
                                                                             shuffle=False)

# create feature and targets tensor for train set. As you remember we need variable to accumulate gradients. Therefore first we create tensor, then we will create variable
featuresTrain = torch.from_numpy(features_train)[:-(features_train.shape[0] % batch_size)]
targetsTrain = torch.from_numpy(targets_train).type(torch.LongTensor)[:-(targets_train.shape[0] % batch_size)] # data type is long

# create feature and targets tensor for test set.
featuresTest = torch.from_numpy(features_test)[:-(features_test.shape[0] % batch_size)]
targetsTest = torch.from_numpy(targets_test).type(torch.LongTensor)[:-(targets_test.shape[0] % batch_size)] # data type is long

# batch_size, epoch and iteration
batch_size = 10
n_iters = 50000
num_epochs = n_iters / (len(features_train) / batch_size)
num_epochs = int(num_epochs)

train = TensorDataset(featuresTrain,targetsTrain)
test = TensorDataset(featuresTest,targetsTest)

# data loader
train_loader = DataLoader(train, batch_size = batch_size, shuffle = False)
test_loader = DataLoader(test, batch_size = batch_size, shuffle = False)

In [0]:
# Create RNN Model
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()
        
        # Number of hidden dimensions
        self.hidden_dim = hidden_dim
        
        # Number of hidden layers
        self.layer_dim = layer_dim
        
        # RNN
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True, nonlinearity='tanh')
        
        # Readout layers
        self.fc = nn.Linear(hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        # print(x.shape)
        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
            
        # One time step
        out, hn = self.rnn(x, h0)
        # print('size of vector leaving rnn', out.size())
        # print('size of weird reshape', out[:, -1, :].size())
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Create RNN
input_dim = features_train.shape[2]    # input dimension - number of routers
hidden_dim = 100  # hidden layer dimension
layer_dim = 3     # number of hidden layers
output_dim = 5   # output dimension - number of positions (5 currently)

model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim)
# use_cuda = True
# if use_cuda and torch.cuda.is_available():
#   model.cuda()

# Cross Entropy Loss 
error = nn.CrossEntropyLoss()

# Optimizer
learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [0]:
seq_dim = 10  
loss_list = []
iteration_list = []
accuracy_list = []
count = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        train  = Variable(images)
        # train = train.cuda()
        labels = Variable(labels)
        # labels.cuda()
            
        # Clear gradients
        optimizer.zero_grad()
        
        # Forward propagation
        outputs = model(train)
        
        # Calculate softmax and cross entropy loss
        loss = error(outputs.view(-1, 5), labels.view(100))
        
        # Calculating gradients
        loss.backward()
        
        # Update parameters
        optimizer.step()
        
        count += 1
        
        if count % 100 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for test_images, test_labels in test_loader:
                test_images = Variable(test_images)
                # test_images.cuda()
                test_labels = Variable(test_labels)
                # test_labels.cuda()
                
                # Forward propagation
                outputs = model(images)
                
                # Get predictions from the maximum value
                predicted = torch.max(outputs.data, 2)[1]
                
                # Total number of labels
                total += test_labels.size(1)*test_labels.size(0)
                
                correct += (predicted == test_labels).sum()
                # print('pred:', np.asarray(predicted[0]), '        labels:', np.asarray(np.asarray(test_labels[0])))
                # print('labels:', np.asarray(test_labels[0]))
            
            accuracy = 100 * correct / float(total)
            
            # store loss and iteration
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)
            if count % 200 == 0:
                # Print Loss
                print('Iteration: {}  Loss: {}  Accuracy: {} %'.format(count, loss.data.item(), accuracy))

In [0]:
# visualization loss 
plt.plot(iteration_list,loss_list)
plt.xlabel("Number of iteration")
plt.ylabel("Loss")
plt.title("RNN: Loss vs Number of iteration")
plt.show()

# visualization accuracy 
plt.plot(iteration_list,accuracy_list,color = "red")
plt.xlabel("Number of iteration")
plt.ylabel("Accuracy")
plt.title("RNN: Accuracy vs Number of iteration")
plt.savefig('graph.png')
plt.show()